# Example 3: Propagation of H2O vibration under polynomial PES

| run type      | wavefunction | backend | Basis  | steps |
| ---           | ---          | ---     | ---    | ---   |
| propagation | MPS-SM | Numpy   | HO-DVR | 5000    |

## 1. Import modules

- Required in **any** calculations

In [1]:
from pytdscf import BasInfo, Model, Simulator

## 2. Set DVR primitive basis

In [2]:
from math import sqrt

from discvar import HarmonicOscillator

from pytdscf import units
from pytdscf.potentials.h2o_potential import k_orig
from pytdscf.dvr_operator_cls import TensorOperator
from pytdscf.hamiltonian_cls import TensorHamiltonian

import numpy as np
import sympy
from pympo import AssignManager, OpSite, SumOfProducts

backend = "numpy"

freqs = [
    sqrt(k_orig[(1, 1)]),
    sqrt(k_orig[(2, 2)]),
    sqrt(k_orig[(3, 3)]),
]  # a.u.
freqs

nprims = [9, 9, 9]  # Number of primitive basis

basis = [
    HarmonicOscillator(nprim, omega, units="a.u.")
    for nprim, omega in zip(nprims, freqs, strict=False)
]
ndim = len(basis)  # Number of degree of freedom, H2O has 3 DOFs
basinfo = BasInfo([basis])  # Set basis information object

print(basinfo)

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.poly_HO_FBR).
Here one define $n_p$ = 9, $N_p$ = 9. (Standard Method)

**NOTE**

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH).  Only when using a laptop, MCTDH may be better. (RAM required in MCTDH is smaller than SM.)

## 3. Set Hamiltonian (Polynomial Function)

Here, one uses pre-calculated Polyonimal PES and DMS.

### Setup one particle operator

In [3]:
q1_list = [np.array(basis_i.get_grids()) for basis_i in basis]
q2_list = [q1_int**2 for q1_int in q1_list]
q3_list = [q1_int**3 for q1_int in q1_list]
q4_list = [q1_int**4 for q1_int in q1_list]
q0_list = [q1_int**0 for q1_int in q1_list]
p2_list = [basis_i.get_2nd_derivative_matrix_dvr() for basis_i in basis]

p2_ops = [OpSite(f"p^2_{i+1}", i, value=p2_list[i]) for i in range(0, ndim)]
q4_ops = [OpSite(f"q^4_{i+1}", i, value=q4_list[i]) for i in range(0, ndim)]
q3_ops = [OpSite(f"q^3_{i+1}", i, value=q3_list[i]) for i in range(0, ndim)]
q2_ops = [OpSite(f"q^2_{i+1}", i, value=q2_list[i]) for i in range(0, ndim)]
q1_ops = [OpSite(f"q_{i+1}", i, value=q1_list[i]) for i in range(0, ndim)]
q0_ops = [OpSite(f"q^0_{i+1}", i, value=q0_list[i]) for i in range(0, ndim)]

qn_list = [q0_ops, q1_ops, q2_ops, q3_ops, q4_ops]

In [4]:
from collections import Counter

k_tensor = np.zeros((5, 5, 5))
key_list = list(k_orig.keys())

for key in key_list:
    count = Counter(key)
    k_tensor[count[1]][count[2]][count[3]] = k_orig[key]

k_symbol = [[[sympy.symbols("k_{" + "1"*i + "2"*j + "3"*k + "}") for k in range(5)]
                                     for j in range(5)]
                                     for i in range(5)]
subs = {}

for i in range(5):
    for j in range(5):
        for k in range(5):
            subs[k_symbol[i][j][k]] = k_tensor[i][j][k]

### Setup Kinetic and Potential Operator

In [5]:
from scipy.special import factorial

pot_sop = SumOfProducts()

for i in range(3):
    pot_sop += -0.5 * p2_ops[i]

for i in range(5):
    for j in range(5):
        for k in range(5):
            if i == 0 and j == 0 and k == 0:
                continue
            if k_tensor[i][j][k] == 0:
                continue
            pot_sop += k_symbol[i][j][k] * qn_list[i][0] * qn_list[j][1] * qn_list[k][2] / factorial(i) / factorial(j) / factorial(k)

pot_sop = pot_sop.simplify()
pot_sop.symbol

0.0416666666666667*k_{1111}*q^4_1*q^0_2*q^0_3 + 0.166666666666667*k_{1112}*q^3_1*q_2*q^0_3 + 0.166666666666667*k_{111}*q^3_1*q^0_2*q^0_3 + 0.25*k_{1122}*q^2_1*q^2_2*q^0_3 + 0.5*k_{112}*q^2_1*q_2*q^0_3 + 0.25*k_{1133}*q^2_1*q^0_2*q^2_3 + 0.5*k_{11}*q^2_1*q^0_2*q^0_3 + 0.166666666666667*k_{1222}*q_1*q^3_2*q^0_3 + 0.5*k_{122}*q_1*q^2_2*q^0_3 + 0.5*k_{1233}*q_1*q_2*q^2_3 + 0.5*k_{133}*q_1*q^0_2*q^2_3 + 0.0416666666666667*k_{2222}*q^0_1*q^4_2*q^0_3 + 0.166666666666667*k_{222}*q^0_1*q^3_2*q^0_3 + 0.25*k_{2233}*q^0_1*q^2_2*q^2_3 + 0.5*k_{22}*q^0_1*q^2_2*q^0_3 + 0.5*k_{233}*q^0_1*q_2*q^2_3 + 0.0416666666666667*k_{3333}*q^0_1*q^0_2*q^4_3 + 0.5*k_{33}*q^0_1*q^0_2*q^2_3 - 0.5*p^2_1 - 0.5*p^2_2 - 0.5*p^2_3

### Setup MPO

In [6]:
am_pot = AssignManager(pot_sop)
am_pot.assign()
display(*am_pot.Wsym)
W_prod = sympy.Mul(*am_pot.Wsym)
print(*[f"W{i}" for i in range(am_pot.ndim)], "=")
display(W_prod[0].expand())
pot_mpo = am_pot.numerical_mpo(subs=subs)

2025-04-30 12:44:45.489 | INFO     | pympo.bipartite:assign:286 - assigned 1/3
2025-04-30 12:44:45.507 | INFO     | pympo.bipartite:assign:286 - assigned 2/3
2025-04-30 12:44:45.515 | INFO     | pympo.bipartite:assign:286 - assigned 3/3


Matrix([[1, q_1, q^0_1, q^3_1, p^2_1, q^4_1, q^2_1]])

Matrix([
[                                                                                     0,                                                        0, -0.5*p^2_2, 1,     0],
[                                  0.166666666666667*k_{1222}*q^3_2 + 0.5*k_{122}*q^2_2,                     0.5*k_{1233}*q_2 + 0.5*k_{133}*q^0_2,          0, 0,     0],
[0.0416666666666667*k_{2222}*q^4_2 + 0.166666666666667*k_{222}*q^3_2 + 0.5*k_{22}*q^2_2, 0.25*k_{2233}*q^2_2 + 0.5*k_{233}*q_2 + 0.5*k_{33}*q^0_2,          0, 0, q^0_2],
[                      0.166666666666667*k_{1112}*q_2 + 0.166666666666667*k_{111}*q^0_2,                                                        0,          0, 0,     0],
[                                                                                     0,                                                        0,       -0.5, 0,     0],
[                                                     0.0416666666666667*k_{1111}*q^0_2,                                                     

Matrix([
[                            q^0_3],
[                            q^2_3],
[                                1],
[                       -0.5*p^2_3],
[0.0416666666666667*k_{3333}*q^4_3]])

W0 W1 W2 =


0.0416666666666667*k_{1111}*q^4_1*q^0_2*q^0_3 + 0.166666666666667*k_{1112}*q^3_1*q_2*q^0_3 + 0.166666666666667*k_{111}*q^3_1*q^0_2*q^0_3 + 0.25*k_{1122}*q^2_1*q^2_2*q^0_3 + 0.5*k_{112}*q^2_1*q_2*q^0_3 + 0.25*k_{1133}*q^2_1*q^0_2*q^2_3 + 0.5*k_{11}*q^2_1*q^0_2*q^0_3 + 0.166666666666667*k_{1222}*q_1*q^3_2*q^0_3 + 0.5*k_{122}*q_1*q^2_2*q^0_3 + 0.5*k_{1233}*q_1*q_2*q^2_3 + 0.5*k_{133}*q_1*q^0_2*q^2_3 + 0.0416666666666667*k_{2222}*q^0_1*q^4_2*q^0_3 + 0.166666666666667*k_{222}*q^0_1*q^3_2*q^0_3 + 0.25*k_{2233}*q^0_1*q^2_2*q^2_3 + 0.5*k_{22}*q^0_1*q^2_2*q^0_3 + 0.5*k_{233}*q^0_1*q_2*q^2_3 + 0.0416666666666667*k_{3333}*q^0_1*q^0_2*q^4_3 + 0.5*k_{33}*q^0_1*q^0_2*q^2_3 - 0.5*p^2_1 - 0.5*p^2_2 - 0.5*p^2_3

### Setup Hamiltonian

In [7]:
potential = [
    [
        {
            (tuple((i, i) for i in range(0, ndim))): TensorOperator(
                mpo=pot_mpo
            )
        }
    ]
]  # key is ((0,0), 1, 2, ..., ndim-1)

H = TensorHamiltonian(
    ndof=len(basis), potential=potential, kinetic=None, backend=backend
)

operators = {"hamiltonian": H}

## 4. Set wavefunction (MPS) and All Model

- `m_aux_max` is MPS bond dimension (maximum of auxiliary index $\tau_p$)


In [8]:
model = Model(basinfo, operators)
model.m_aux_max = 9

## 5. Execute Calculation

- time step width is defined by `stepsize`=0.05 fs

In this calculation, one runs

- Real-time propagation

- Restart from $\hat{\mu}|\Psi_{\rm GS}\rangle$ wavefunction. (restart file suffix is `_dipole`)

F.Y.I., See also [documentation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.const_cls.Const.set_runtype)


**NOTE**

- Runtype cannnot rebind. If you change runtype, you should restart the kernel.

- JAX is better when simulating more large systems. (f>6, m>10)

- If `AVG Krylov iteration` in the log file is much larger than 5, you should set smaller timestep.

In [9]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, backend="numpy")
simulator.propagate(
    maxstep=10000,
    stepsize=0.05,
    restart=True,
    savefile_ext="_prop",
    loadfile_ext="_dipole",
)  # i.e., 500 fs

2025-04-30 12:44:46,552 - INFO:main.pytdscf._const_cls - 
     ____     __________   .____ ____   _____
    / _  |   /__  __/ _ \ / ___ / _  \ / ___/
   / /_) /_  __/ / / / ||/ /__ / / )_// /__
  /  ___/ / / / / / / / |.__  / |  __/ ___/
 /  /  / /_/ / / / /_/ /___/ /| \_/ / /
/__/   \__, /_/ /_____/_____/ \____/_/
      /____/

2025-04-30 12:44:46,554 - INFO:main.pytdscf._const_cls - Log file is ./h2o_polynomial_prop/main.log
2025-04-30 12:44:46,555 - INFO:main.pytdscf.simulator_cls - Set integral of DVR basis
2025-04-30 12:44:46,573 - INFO:main.pytdscf.simulator_cls - Set initial wave function (DVR basis)
2025-04-30 12:44:46,621 - INFO:main.pytdscf.simulator_cls - Wave function is loaded from wf_h2o_polynomial_dipole.pkl
2025-04-30 12:44:46,649 - INFO:main.pytdscf.simulator_cls - Wave function is saved in wf_h2o_polynomial_prop.pkl
2025-04-30 12:44:46,657 - INFO:main.pytdscf.simulator_cls - Start initial step    0.000 [fs]
2025-04-30 12:44:47,058 - INFO:main.pytdscf.simulator_cls - E

(0.020896110037573204, <pytdscf.wavefunction.WFunc at 0x117c12990>)

## 6. Check Log file
See `h2o_polynomial_prop/main.log`, which is defined as `jobname`.

In [10]:
!tail h2o_polynomial_prop/main.log

| autocorr:  0.6236-0.7767i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.700 | elapsed[sec]:   706.90 
| autocorr:  0.5543-0.8274i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.750 | elapsed[sec]:   706.96 
| autocorr:  0.4809-0.8720i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.800 | elapsed[sec]:   707.03 
| autocorr:  0.4039-0.9100i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.850 | elapsed[sec]:   707.10 
| autocorr:  0.3240-0.9413i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.900 | elapsed[sec]:   707.16 
Saved wavefunction  499.950 [fs]
| autocorr:  0.2417-0.9656i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  499.950 | elapsed[sec]:   707.22 
End  9999 step; propagated  499.950 [fs]; AVG Krylov iteration: 5.00
End simulation and save wavefunction
Wave function is saved in wf_h2o_polynomial_prop.pkl


/usr/local/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
